In [ ]:
!pip install transformers torch librosa pydub google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client #Install all the required libraries

In [ ]:
from google.colab import files
uploaded = files.upload()# Now choose your .json file and click upload

In [ ]:
from google.oauth2 import service_account #Authenticate your service account which you have created
from googleapiclient.discovery import build

SERVICE_ACCOUNT_FILE = "meeting-summarization-tool-00e383a89444.json"
SCOPES = ["https://www.googleapis.com/auth/drive"]

creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
drive_service = build("drive", "v3", credentials=creds)
print("Successfully authenticated google drive API") #Your API should be successfully authenticated

In [ ]:
import os
import torch
import librosa
import textwrap
from pydub import AudioSegment
from google.colab import files
from transformers import pipeline, WhisperProcessor, WhisperForConditionalGeneration
from googleapiclient.http import MediaFileUpload

# Create necessary folders/directories e.g uploads and summaries
UPLOAD_FOLDER = '/content/uploaded_files'
SUMMARY_FOLDER = '/content/meeting_summaries'
os.makedirs(UPLOAD_FOLDER, exist_ok=True) #If the folder already doesn't exist it will be created successfully
os.makedirs(SUMMARY_FOLDER, exist_ok=True)#If the folder already doesn't exist it will be created successfully

ALLOWED_EXTENSIONS = {'mp3', 'mp4', 'wav'} #The allowed extensions for the upload file.

# Set Drive Folder for storage
DRIVE_FOLDER_ID = "1v-rL90zvmXL8_OlRI_gtRcW1wePXTkdV"  # Make sure to replace your Drive ID

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def extract_audio(video_path):# Extract audio from the provided input files
    try:
        video = AudioSegment.from_file(video_path, format="mp4")
        audio_path = os.path.join(UPLOAD_FOLDER, 'extracted_audio.wav')
        video.export(audio_path, format="wav")
        return audio_path
    except Exception as e:
        print(f"Error extracting audio: {e}")
        return None

def transcribe_audio(audio_path):#Transcribe speech from audo files using Whisper model.
    try:
        model_id = "openai/whisper-large-v2"
        processor = WhisperProcessor.from_pretrained(model_id)
        model = WhisperForConditionalGeneration.from_pretrained(model_id)

        audio, _ = librosa.load(audio_path, sr=16000)
        input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features
        predicted_ids = model.generate(input_features)
        transcript = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        return transcript
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

def generate_summary(text):#Generates concise summary
    try:
        # Use multiple summarization models to improve results
        bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        t5_summarizer = pipeline("summarization", model="t5-base")

        # Generate summaries
        bart_summary = bart_summarizer(text, max_length=150, min_length=60, do_sample=False)[0]['summary_text']
        t5_summary = t5_summarizer(text, max_length=100, min_length=50, do_sample=False)[0]['summary_text']

        # Combine results and extract key points
        combined_summary = bart_summary + " " + t5_summary
        summary_lines = combined_summary.split('. ')

        # Post-process: convert into actionable bullet points (max 10)
        bullet_points = []
        for line in summary_lines:
            if len(line.strip()) > 0:
                bullet_points.append(f"- {line.strip()}")

        # Ensure a max of 10 bullet points
        return "\n".join(bullet_points[:10])

    except Exception as e:
        print(f"Error generating summary: {e}")
        return None
def sentiment_analyze(text):#Perform sentiment analysis
    try:
        sentiment_pipeline = pipeline("sentiment-analysis")
        sentiment = sentiment_pipeline(text)[0]
        return f"Sentiment: {sentiment['label']} (Confidence: {sentiment['score']:.2f})"
    except Exception as e:
        print(f"Error performing sentiment analysis: {e}")
        return None

def upload_to_drive(file_path):#Uploading file to the drive.
    try:
        file_metadata = {
            "name": os.path.basename(file_path),
            "parents": [DRIVE_FOLDER_ID]
        }
        media = MediaFileUpload(file_path, mimetype="text/plain")
        file = drive_service.files().create(body=file_metadata, media_body=media, fields="id").execute()
        print(f"File uploaded to Google Drive: {file.get('id')}")
        return file.get("id")
    except Exception as e:
        print(f"Error uploading to Drive: {e}")
        return None

def download_from_drive(file_id, save_path):#Downloads file from the drive.
    try:
        request = drive_service.files().get_media(fileId=file_id)
        with open(save_path, "wb") as f:
            f.write(request.execute())
        print(f" File downloaded: {save_path}")
        return save_path
    except Exception as e:
        print(f"Error downloading from Drive: {e}")
        return None

def save_summary(summary, sentiment, filename):#Saves the summary to the drive folder.
    try:
        summary_path = os.path.join(SUMMARY_FOLDER, f"{filename}_summary.txt")
        with open(summary_path, 'w') as file:
            file.write("=== Meeting Summary ===\n")
            file.write(summary + "\n\n")
            file.write("=== Sentiment Analysis ===\n")
            file.write(sentiment + "\n")

        drive_file_id = upload_to_drive(summary_path)
        return drive_file_id
    except Exception as e:
        print(f"Error saving summary: {e}")
        return None

uploaded = files.upload()
for filename in uploaded.keys():
    file_path = os.path.join(UPLOAD_FOLDER, filename)
    with open(file_path, 'wb') as f:
        f.write(uploaded[filename])
    print(f"File uploaded: {filename}")

    if filename.endswith('.mp4'):
        audio_path = extract_audio(file_path)
    else:
        audio_path = file_path

    if audio_path:
        transcript = transcribe_audio(audio_path)
        if transcript:
            summary = generate_summary(transcript)
            sentiment = sentiment_analyze(transcript)
            if summary and sentiment:
                drive_file_id = save_summary(summary, sentiment, filename)
                if drive_file_id:
                    print(f"Summary saved & uploaded to Drive: {drive_file_id}")#The summary would be uploaded to your drive ID.
                else:
                    print("Error saving summary.")
            else:
                print("Error generating summary or sentiment analysis.")
        else:
            print("Error transcribing audio.")
    else:
        print("Error extracting audio.")